# Import Library

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import math
import os
import time
import sys
import re
from string import punctuation
from nltk import word_tokenize
from collections import Counter
from numpy import linalg as LA
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Import Database

In [2]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/data Soap Indo.xlsx')
data

,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",NaN,<p>paracetamol</p> <p>ranitidin</p> <p>imbos...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,NaN,<p>R/ Lycalvit syr 1x 1 cth&nbsp;</p>,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>ISK</p> <p>Post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>ISK</p> <p>Post rawat inap&nbsp;</p>
3,<p>imunisasi</p>,NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",NaN
4,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",NaN,NaN,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Obs febris hr ke 3 ec&nbsp;</p>
...,...,...,...,...,...
126478,<p>IMUNISASI HEPATITIS</p> <p>Anti HBs Ag (-)...,NaN,<p>Hepavax B/ Engerix B</p>,"<p>Suhu : C,Nadi : 97 x/menit,Respirasi : x/me...",<p>Vaksinasi HEp B</p>
126479,"<p>BAWAH DADA KIRI SAKIT,KADANG KADANG NYERI P...",NaN,<p>Lancid mg 30 no X<br />S2ddI</p> <p>Aldisa...,"<p>Suhu : C,Nadi : 79 x/menit,Respirasi : x/me...",<p>Dyspepsia</p>
126480,"<p>-telinga kiri ndenging sekitar 3 hari, kura...",NaN,<p>Carbogliserin s.3.d.d.gtt IV AS Lg I</p>,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>AS cerumen prop</p> <p>&nbsp;</p>
126481,"<p>TELINGA KANAN KELUAR CAIRAN, 4 hari. batpil...",<p>evakuasi discharge AD</p>,<p>Claneksi syr s.3.d.d. Ctakar I Lg I</p> <p...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>AD OMA perforasi</p>


In [3]:
data_index = data[['FS_ANAMNESA', 'FS_DIAGNOSA']]

In [4]:
typo = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/Check Typo.xlsx')
typo

,kata_typo,kata_benar
0,peerut,perut
1,jln,jalan
2,kakan,kanan
3,umroih,umroh
4,kleuhan,keluhan
...,...,...
1285,shdh,sudah
1286,muntahdemampusing,muntah demam pusing
1287,gejla,gejala
1288,ebih,lebih


# Preprocessing

## Missing Value

In [5]:
# check nan
percent_missing = data_index.isnull().sum() * 100 / len(data)
percent_missing

FS_ANAMNESA    3.335626
FS_DIAGNOSA    1.093428
dtype: float64

In [6]:
index = data_index[data_index.isna().any(axis=1)]
index

,FS_ANAMNESA,FS_DIAGNOSA
3,<p>imunisasi</p>,NaN
82,NaN,<p>36 KARIES</p>
84,NaN,<p>36 KARIES</p>
110,NaN,NaN
180,NaN,<p>48 PULPITIS</p>
...,...,...
126308,NaN,<p>ISPA</p> <p>Dyspepsia</p>
126309,NaN,<p>Dyspepsia</p>
126323,NaN,<p>Gaswtropathy drug induced</p>
126360,<p>KONTROL RUTIN DM</p> <p>GDS 302</p>,NaN


### Drop Missing Value

In [7]:
data = data.dropna(subset = ['FS_ANAMNESA', 'FS_DIAGNOSA'])
data = data.reset_index(drop=True)

In [8]:
data_index = data[['FS_ANAMNESA', 'FS_DIAGNOSA']]

In [9]:
data_id = data_index.index

In [10]:
data_id

RangeIndex(start=0, stop=120986, step=1)

In [11]:
data_index = data_index.iloc[:1000]
data_index

,FS_ANAMNESA,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,<p>ISK</p> <p>Post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,<p>ISK</p> <p>Post rawat inap&nbsp;</p>
3,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",<p>Obs febris hr ke 3 ec&nbsp;</p>
4,<p>kontrol</p>,<p>kontrol BKB alergi&nbsp;</p>
...,...,...
995,"<p>rujukan dengan ca mamae,PA terlampir</p>",<p>Ca mammae pro SS I</p>
996,"<p>gatal TENGGOROKAN, BATUK tidak berdahak&nbs...",<p>faringitis alergi&nbsp;</p>
997,<p>RENCANA UMROH 29 SEPT 19..KELUHAN (-)</p>,<p>PRO VAKSIN</p>
998,<p>telinga nyeri</p>,<p>AD OMA hiperemi</p> <p>AD Otitis eksterna</p>


## Lowercase

In [12]:
data_index_anam = data_index['FS_ANAMNESA'].apply(lambda x: x.lower())
data_index_diag = data_index['FS_DIAGNOSA'].apply(lambda x: x.lower())

In [13]:
data_index_anam

0      <p>demam 2 hari, pusing, ma/mi baik</p>  <p>le...
1                             <p>kontrol post opname</p>
2                             <p>kontrol post opname</p>
3       <p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>
4                                         <p>kontrol</p>
                             ...                        
995          <p>rujukan dengan ca mamae,pa terlampir</p>
996    <p>gatal tenggorokan, batuk tidak berdahak&nbs...
997         <p>rencana umroh 29 sept 19..keluhan (-)</p>
998                                 <p>telinga nyeri</p>
999                <p>telinga tersumbat, batuk pilek</p>
Name: FS_ANAMNESA, Length: 1000, dtype: object

In [14]:
df_lower = pd.concat([data_index_anam, data_index_diag], axis=1)
df_lower

,FS_ANAMNESA,FS_DIAGNOSA
0,"<p>demam 2 hari, pusing, ma/mi baik</p> <p>le...",<p>febris hari 2</p>
1,<p>kontrol post opname</p>,<p>isk</p> <p>post rawat inap&nbsp;</p>
2,<p>kontrol post opname</p>,<p>isk</p> <p>post rawat inap&nbsp;</p>
3,"<p>panas ,&nbsp;3 hari, miuntah, pilek&nbsp;</p>",<p>obs febris hr ke 3 ec&nbsp;</p>
4,<p>kontrol</p>,<p>kontrol bkb alergi&nbsp;</p>
...,...,...
995,"<p>rujukan dengan ca mamae,pa terlampir</p>",<p>ca mammae pro ss i</p>
996,"<p>gatal tenggorokan, batuk tidak berdahak&nbs...",<p>faringitis alergi&nbsp;</p>
997,<p>rencana umroh 29 sept 19..keluhan (-)</p>,<p>pro vaksin</p>
998,<p>telinga nyeri</p>,<p>ad oma hiperemi</p> <p>ad otitis eksterna</p>


## Remove Tag HTML

In [15]:
# remove tag html

df_tag_removed = df_lower.replace(r'<[^<>]*>', '', regex=True)
df_tag_removed

,FS_ANAMNESA,FS_DIAGNOSA
0,"demam 2 hari, pusing, ma/mi baik lemas(-)",febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp;
2,kontrol post opname,isk post rawat inap&nbsp;
3,"panas ,&nbsp;3 hari, miuntah, pilek&nbsp;",obs febris hr ke 3 ec&nbsp;
4,kontrol,kontrol bkb alergi&nbsp;
...,...,...
995,"rujukan dengan ca mamae,pa terlampir",ca mammae pro ss i
996,"gatal tenggorokan, batuk tidak berdahak&nbsp;",faringitis alergi&nbsp;
997,rencana umroh 29 sept 19..keluhan (-),pro vaksin
998,telinga nyeri,ad oma hiperemi ad otitis eksterna


## Remove Punctuation

In [16]:
# remove punct except between number

df_punct_exc_number = df_tag_removed.replace(r'(?<!\d)[.,;:+](?!\d)', '', regex=True)
df_punct_exc_number

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma/mi baik lemas(-),febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp
2,kontrol post opname,isk post rawat inap&nbsp
3,panas &nbsp;3 hari miuntah pilek&nbsp,obs febris hr ke 3 ec&nbsp
4,kontrol,kontrol bkb alergi&nbsp
...,...,...
995,rujukan dengan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak&nbsp,faringitis alergi&nbsp
997,rencana umroh 29 sept 19.keluhan (-),pro vaksin
998,telinga nyeri,ad oma hiperemi ad otitis eksterna


In [17]:
df_punct_exc_number = df_punct_exc_number.replace(r'[(_)!?@/=;:#$*^~{}\-]', ' ', regex=True)
df_punct_exc_number

,FS_ANAMNESA,FS_DIAGNOSA
0,demam 2 hari pusing ma mi baik lemas,febris hari 2
1,kontrol post opname,isk post rawat inap&nbsp
2,kontrol post opname,isk post rawat inap&nbsp
3,panas &nbsp 3 hari miuntah pilek&nbsp,obs febris hr ke 3 ec&nbsp
4,kontrol,kontrol bkb alergi&nbsp
...,...,...
995,rujukan dengan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak&nbsp,faringitis alergi&nbsp
997,rencana umroh 29 sept 19.keluhan,pro vaksin
998,telinga nyeri,ad oma hiperemi ad otitis eksterna


In [18]:
df_punct_exc_number = df_punct_exc_number.replace(r'[.,;:+-?!(_)@/#$*^~{}](?!\d\w)',' ', regex=True)
df_punct_exc_number

,FS_ANAMNESA,FS_DIAGNOSA
0,demam hari pusing ma mi baik lemas,febris hari
1,kontrol post opname,isk post rawat inap&nbsp
2,kontrol post opname,isk post rawat inap&nbsp
3,panas &nbsp hari miuntah pilek&nbsp,obs febris hr ke ec&nbsp
4,kontrol,kontrol bkb alergi&nbsp
...,...,...
995,rujukan dengan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak&nbsp,faringitis alergi&nbsp
997,rencana umroh sept keluhan,pro vaksin
998,telinga nyeri,ad oma hiperemi ad otitis eksterna


## Remove Stopwords

### FS_ANAMNESA

In [19]:
# remove stopwords FS_ANAMNESA

df_remove_sw = df_punct_exc_number['FS_ANAMNESA']
dict_doc_anam = []

def filtered(doc):
  open_stopwords = open('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/stopwords.txt', 'r')
  stopwords = open_stopwords.read()
  stopwords = stopwords.split("\n")
  word_tokens = word_tokenize(doc)
  filtered_sentence = [w for w in word_tokens if not w in stopwords]
  return ' '.join(filtered_sentence)

def remove_stopwords(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    dict_doc_anam.append(data)
  return dict_doc_anam

remove_stopwords(df_remove_sw)

['demam hari pusing ma mi baik lemas',
 'kontrol post opname',
 'kontrol post opname',
 'panas hari miuntah pilek',
 'kontrol',
 'batuk panas pilek',
 'batuk sering kambuh',
 'batuk pilek demam kemerahan badan',
 'batuk sesak napas',
 'rujukan klinik laras hati gigi kiri atas berlubang',
 'melanjutkan perawatan gigi sebelah kiri atas',
 'melanjutkan perawatan',
 'pusing lemas batuk demam hari dahak warna kuning kental paracetamol tidak membaik',
 'minta surat keterangan sehat bebas narkoba',
 'kontrol evaluasi bmp',
 'kontrol',
 'kontrol et konsultasi hasil laborat',
 'kontrol',
 'kontrol et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kontrol te',
 'kontrol pembesaran kelenjar getah bening leher',
 'g p ab hpm kellkadang nyeri perut bagian bawah',
 'kadang kenceng kencenghpm juni mg',
 'g p a hpht 20 oeriksa hamil t a k',
 'g p a hpht periksa kehamilan',
 'g p a hpht + mggu kenceng capek',
 'kontrol hamil g p a hpht',
 'g p ab hpm a

In [20]:
df_remove_sw_an = pd.Series(dict_doc_anam)
df_remove_sw_an

0          demam hari pusing ma mi baik lemas
1                         kontrol post opname
2                         kontrol post opname
3                    panas hari miuntah pilek
4                                     kontrol
                        ...                  
995              rujukan ca mamaepa terlampir
996    gatal tenggorokan batuk tidak berdahak
997                        umroh sept keluhan
998                             telinga nyeri
999             telinga tersumbat batuk pilek
Length: 1000, dtype: object

### FS_DIAGNOSA

In [21]:
# remove stopwords FS_ANAMNESA

df_remove_sw = df_punct_exc_number['FS_DIAGNOSA']
dict_doc_diag = []

def filtered(doc):
  open_stopwords = open('/content/drive/MyDrive/Colab Notebooks/STBI/Sesudah UTS/stopwords.txt', 'r')
  stopwords = open_stopwords.read()
  stopwords = stopwords.split("\n")
  word_tokens = word_tokenize(doc)
  filtered_sentence = [w for w in word_tokens if not w in stopwords]
  return ' '.join(filtered_sentence)

def remove_stopwords(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    dict_doc_diag.append(data)
  return dict_doc_diag

remove_stopwords(df_remove_sw)

['febris hari',
 'isk post rawat inap',
 'isk post rawat inap',
 'obs febris ec',
 'kontrol bkb alergi',
 'rfa',
 'bkb ec alergi susp asma',
 'scarlet fever',
 'asma bronkhiale serang',
 'gigi nv gingival polyp',
 'gigi nv',
 'nv',
 'common cold',
 'sehat bebas narkoba',
 'et',
 'thrombositosis esensial limfadenopati reaktif',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'mpd esential thrombositosis',
 'et',
 'te',
 'tromvbositosis esensial',
 'te',
 'mg',
 'hamil mg',
 'hamil mg',
 'hamil mg preskep',
 'hamil mg vitamin diteruskan',
 'hamil mg',
 'hamil mg edd usg 20',
 'konsultasi ia turunkan bb suami pernah minum cc post op varicocele bulan juni permata hati',
 'calculus penundaan haid',
 'calculus',
 'dermatitis atopik',
 'dermatitis atopik',
 'sun burn nbspdermatitis foto kontak alergika',
 'dki nbsppityriasis alba',
 'pitted keratolysis',
 'dermatitis atopik',
 'pompholyx',
 'tinea korporis nbspdd dermatitis numular',
 'tinea korporis nbspdd dermatitis numular',
 'tin

In [22]:
df_remove_sw_diag = pd.Series(dict_doc_diag)
df_remove_sw_diag

0                       febris hari
1               isk post rawat inap
2               isk post rawat inap
3                     obs febris ec
4                kontrol bkb alergi
                   ...             
995              ca mammae pro ss i
996               faringitis alergi
997                      pro vaksin
998    oma hiperemi otitis eksterna
999            ads serumen rhinitis
Length: 1000, dtype: object

In [23]:
df_remove_sw = pd.concat([df_remove_sw_an, df_remove_sw_diag], axis=1)
df_remove_sw

,0,1
0,demam hari pusing ma mi baik lemas,febris hari
1,kontrol post opname,isk post rawat inap
2,kontrol post opname,isk post rawat inap
3,panas hari miuntah pilek,obs febris ec
4,kontrol,kontrol bkb alergi
...,...,...
995,rujukan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak,faringitis alergi
997,umroh sept keluhan,pro vaksin
998,telinga nyeri,oma hiperemi otitis eksterna


In [24]:
df_remove_sw.columns =['FS_ANAMNESA', 'FS_DIAGNOSA']

In [25]:
df_remove_sw

,FS_ANAMNESA,FS_DIAGNOSA
0,demam hari pusing ma mi baik lemas,febris hari
1,kontrol post opname,isk post rawat inap
2,kontrol post opname,isk post rawat inap
3,panas hari miuntah pilek,obs febris ec
4,kontrol,kontrol bkb alergi
...,...,...
995,rujukan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak,faringitis alergi
997,umroh sept keluhan,pro vaksin
998,telinga nyeri,oma hiperemi otitis eksterna


## Remove Multiple Whitespace

In [26]:
# remove double whitespace
df_content_anam = df_remove_sw['FS_ANAMNESA'].replace(r'\s\s+', ' ')
df_content_anam

0          demam hari pusing ma mi baik lemas
1                         kontrol post opname
2                         kontrol post opname
3                    panas hari miuntah pilek
4                                     kontrol
                        ...                  
995              rujukan ca mamaepa terlampir
996    gatal tenggorokan batuk tidak berdahak
997                        umroh sept keluhan
998                             telinga nyeri
999             telinga tersumbat batuk pilek
Name: FS_ANAMNESA, Length: 1000, dtype: object

In [27]:
# remove double whitespace
df_content_diag = df_remove_sw.FS_DIAGNOSA.replace(r'\s\s+', ' ')
df_content_diag

0                       febris hari
1               isk post rawat inap
2               isk post rawat inap
3                     obs febris ec
4                kontrol bkb alergi
                   ...             
995              ca mammae pro ss i
996               faringitis alergi
997                      pro vaksin
998    oma hiperemi otitis eksterna
999            ads serumen rhinitis
Name: FS_DIAGNOSA, Length: 1000, dtype: object

In [28]:
df_content = pd.concat([df_content_anam, df_content_diag], axis=1)
df_content

,FS_ANAMNESA,FS_DIAGNOSA
0,demam hari pusing ma mi baik lemas,febris hari
1,kontrol post opname,isk post rawat inap
2,kontrol post opname,isk post rawat inap
3,panas hari miuntah pilek,obs febris ec
4,kontrol,kontrol bkb alergi
...,...,...
995,rujukan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak,faringitis alergi
997,umroh sept keluhan,pro vaksin
998,telinga nyeri,oma hiperemi otitis eksterna


## Replacement Typo Word

#### FS_ANAMNESA

In [29]:
#replacement typo word
content_anam = []

def filtered(doc):
  word_tokens = word_tokenize(doc)
  for x in range(len(word_tokens)):
    for y in range(len(typo)):
      if word_tokens[x] == typo['kata_typo'][y]:
        word_tokens[x] = typo['kata_benar'][y]
      else:
        continue
  return ' '.join(word_tokens)

def replacement(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    content_anam.append(data)
  return content_anam

In [30]:
replacement(df_content['FS_ANAMNESA'])

['demam hari pusing makan minum baik lemas',
 'kontrol post opname',
 'kontrol post opname',
 'panas hari miuntah pilek',
 'kontrol',
 'batuk panas pilek',
 'batuk sering kambuh',
 'batuk pilek demam kemerahan badan',
 'batuk sesek nafas',
 'rujukan klinik laras hati gigi kiri atas berlubang',
 'melanjutkan perawatan gigi sebelah kiri atas',
 'melanjutkan perawatan',
 'pusing lemas batuk demam hari dahak warna kuning kental paracetamol tidak membaik',
 'minta surat keterangan sehat bebas narkoba',
 'kontrol evaluasi bmp',
 'kontrol',
 'kontrol et konsultasi hasil laborat',
 'kontrol',
 'kontrol et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kontrol te',
 'kontrol pembesaran kelenjar getah bening leher',
 'g p ab hpm kellkadang nyeri perut bagian bawah',
 'kadang kenceng kencenghpm juni mg',
 'g p a hpht 20 oeriksa hamil t a k',
 'g p a hpht periksa kehamilan',
 'g p a hpht + minggu kenceng capek',
 'kontrol hamil g p a hpht',
 'g p 

### FS_DIAGNOSA

In [31]:
#replacement typo word
content_diag = []

def filtered(doc):
  word_tokens = word_tokenize(doc)
  for x in range(len(word_tokens)):
    for y in range(len(typo)):
      if word_tokens[x] == typo['kata_typo'][y]:
        word_tokens[x] = typo['kata_benar'][y]
      else:
        continue
  return ' '.join(word_tokens)

def replacement(doc):
  for i in doc.index:  
    data = filtered(doc[i])
    content_diag.append(data)
  return content_diag

In [32]:
replacement(df_content['FS_DIAGNOSA'])

['febris hari',
 'isk post rawat inap',
 'isk post rawat inap',
 'obs febris ec',
 'kontrol bkb alergi',
 'rfa',
 'bkb ec alergi susp asma',
 'scarlet fever',
 'asma bronkhiale serang',
 'gigi non vital gingival polyp',
 'gigi non vital',
 'non vital',
 'common cold',
 'sehat bebas narkoba',
 'et',
 'thrombositosis esensial limfadenopati reaktif',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'et',
 'mpd esential thrombositosis',
 'et',
 'te',
 'tromvbositosis esensial',
 'te',
 'mg',
 'hamil mg',
 'hamil mg',
 'hamil mg preskep',
 'hamil mg vitamin diteruskan',
 'hamil mg',
 'hamil mg edd usg 20',
 'konsultasi ia turunkan bb suami pernah minum cc post op varicocele bulan juni permata hati',
 'calculus penundaan haid',
 'calculus',
 'dermatitis atopik',
 'dermatitis atopik',
 'sun burn nbspdermatitis foto kontak alergika',
 'dki nbsppityriasis alba',
 'pitted keratolysis',
 'dermatitis atopik',
 'pompholyx',
 'tinea korporis nbspdd dermatitis numular',
 'tinea korporis nbspdd derma

### Concatenantion

In [33]:
content_anam = pd.Series(content_anam)
content_diag = pd.Series(content_diag)

In [34]:
content_anam

0      demam hari pusing makan minum baik lemas
1                           kontrol post opname
2                           kontrol post opname
3                      panas hari miuntah pilek
4                                       kontrol
                         ...                   
995                rujukan ca mamaepa terlampir
996      gatal tenggorokan batuk tidak berdahak
997                     umroh september keluhan
998                               telinga nyeri
999                   telinga sesek batuk pilek
Length: 1000, dtype: object

In [35]:
df_content = pd.concat([content_anam, content_diag], axis=1)
df_content

,0,1
0,demam hari pusing makan minum baik lemas,febris hari
1,kontrol post opname,isk post rawat inap
2,kontrol post opname,isk post rawat inap
3,panas hari miuntah pilek,obs febris ec
4,kontrol,kontrol bkb alergi
...,...,...
995,rujukan ca mamaepa terlampir,ca mammae pro ss i
996,gatal tenggorokan batuk tidak berdahak,faringitis alergi
997,umroh september keluhan,pro vaksin
998,telinga nyeri,oma hiperemi otitis eksterna


In [36]:
df_content.columns = ['FS_ANAMNESA', 'FS_DIAGNOSA']

# Process

In [37]:
# make a list of all the words in FS_ANAMNESA and FS_DIAGNOSA
word_list_anam = []
word_list_diag = []
for i in df_content.index:
  word_list_anam = word_list_anam + word_tokenize(df_content['FS_ANAMNESA'][i])
  word_list_diag = word_list_diag + word_tokenize(df_content['FS_DIAGNOSA'][i])

In [38]:
word_list_anam

['demam',
 'hari',
 'pusing',
 'makan',
 'minum',
 'baik',
 'lemas',
 'kontrol',
 'post',
 'opname',
 'kontrol',
 'post',
 'opname',
 'panas',
 'hari',
 'miuntah',
 'pilek',
 'kontrol',
 'batuk',
 'panas',
 'pilek',
 'batuk',
 'sering',
 'kambuh',
 'batuk',
 'pilek',
 'demam',
 'kemerahan',
 'badan',
 'batuk',
 'sesek',
 'nafas',
 'rujukan',
 'klinik',
 'laras',
 'hati',
 'gigi',
 'kiri',
 'atas',
 'berlubang',
 'melanjutkan',
 'perawatan',
 'gigi',
 'sebelah',
 'kiri',
 'atas',
 'melanjutkan',
 'perawatan',
 'pusing',
 'lemas',
 'batuk',
 'demam',
 'hari',
 'dahak',
 'warna',
 'kuning',
 'kental',
 'paracetamol',
 'tidak',
 'membaik',
 'minta',
 'surat',
 'keterangan',
 'sehat',
 'bebas',
 'narkoba',
 'kontrol',
 'evaluasi',
 'bmp',
 'kontrol',
 'kontrol',
 'et',
 'konsultasi',
 'hasil',
 'laborat',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol',
 'et',
 'kontrol',
 'kontrol',
 'te',
 'kontrol',
 'kontrol',
 'kontrol',
 'kontrol',
 'te',
 'kontrol',
 'pembesaran',
 'kelenjar',


In [39]:
word_anam = pd.Series(word_list_anam)
word_diag = pd.Series(word_list_diag)

In [40]:
word_anam

0         demam
1          hari
2        pusing
3         makan
4         minum
         ...   
4694      nyeri
4695    telinga
4696      sesek
4697      batuk
4698      pilek
Length: 4699, dtype: object

In [41]:
word_anam = word_anam.drop_duplicates()
word_diag = word_diag.drop_duplicates()

In [42]:
# size of the corpus

global N
N=len(data_index) # 1399

# create tf-idf matrix for documents

## TF-IDF

### FS_ANAMNESA

In [43]:
columns_anam = ['index_terms_anam']
for i in (df_content.index):
  columns_anam.append('doc_'+str(i))

# initiate the tf matrix
tf_matrix_anam = pd.DataFrame(columns=columns_anam)

# fill up the index terms column using the unique words
tf_matrix_anam['index_terms_anam']=word_anam

# initialize with all zeros
tf_matrix_anam.fillna(0, inplace=True)

# create terms as indices
tf_matrix_anam.set_index(['index_terms_anam'], inplace=True)

# fill tf_matrix with the frequency of each term for every docs
count = 0
for j in word_list_anam:
  count += 1
  for k in df_content.index:
    anam = df_content['FS_ANAMNESA'][k] 
    docid = k
    f = anam.count(j)
    if f !=0:
      tf_matrix_anam['doc_'+str(docid)][j] = f
  print("done for word:",j, "at level", count)
print("Weighted term frequency matrix completed.") 

done for word: demam at level 1
done for word: hari at level 2
done for word: pusing at level 3
done for word: makan at level 4
done for word: minum at level 5
done for word: baik at level 6
done for word: lemas at level 7
done for word: kontrol at level 8
done for word: post at level 9
done for word: opname at level 10
done for word: kontrol at level 11
done for word: post at level 12
done for word: opname at level 13
done for word: panas at level 14
done for word: hari at level 15
done for word: miuntah at level 16
done for word: pilek at level 17
done for word: kontrol at level 18
done for word: batuk at level 19
done for word: panas at level 20
done for word: pilek at level 21
done for word: batuk at level 22
done for word: sering at level 23
done for word: kambuh at level 24
done for word: batuk at level 25
done for word: pilek at level 26
done for word: demam at level 27
done for word: kemerahan at level 28
done for word: badan at level 29
done for word: batuk at level 30
done fo

In [44]:
for i in range(0,N):
    tf_matrix_anam['doc_'+str(i)] = tf_matrix_anam['doc_'+str(i)] / max(tf_matrix_anam['doc_'+str(i)])
    
# append a column D at the end of tf
sLength = len(tf_matrix_anam.index)
D = pd.Series(np.zeros(sLength))
tf_matrix_anam = tf_matrix_anam.assign(D=D.values)
# append an IDF column
IDF = pd.Series(np.zeros(sLength))
tf_matrix_anam = tf_matrix_anam.assign(IDF=IDF.values)
# fill the nonzero count at D

In [45]:
tf_matrix_anam

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_992,doc_993,doc_994,doc_995,doc_996,doc_997,doc_998,doc_999,D,IDF
index_terms_anam,,,,,,,,,,,,,,,,,,,,,
demam,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.166667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
hari,0.166667,0.0,0.0,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
pusing,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
makan,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
minum,0.166667,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ngikil,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
bastuk,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
berlubangbengkak,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
"""
    computing term frequecny and inverce-document frequency
    for every index terms. two columns are being added for this purpose.
"""
for indices in range(len(tf_matrix_anam.index)):
  di = sum(tf_matrix_anam.loc[tf_matrix_anam.index[indices]] > 0)
  # update the di values |di|=non zero entries of a term w.r.t a doc
  tf_matrix_anam['D'][tf_matrix_anam.index[indices]] = di
  # IDF computation
  tf_matrix_anam['IDF'][tf_matrix_anam.index[indices]] = math.log(N/di,10)
  
    
"""
    tf-idf computation phase. compute tf-idf for each entry of the data 
"""
global A
A = tf_matrix_anam
for i in range(len(tf_matrix_anam.index)):
  A.loc[A.index[i]][0:N+1] = A.loc[A.index[i]][0:N+1] * A.IDF[i]

### FS_DIAGNOSA

In [47]:
columns_diag = ['index_terms_diag']
for i in (df_content.index):
  columns_diag.append('doc_'+str(i))

# initiate the tf matrix
tf_matrix_diag = pd.DataFrame(columns=columns_diag)

# fill up the index terms column using the unique words
tf_matrix_diag['index_terms_diag']=word_diag

# initialize with all zeros
tf_matrix_diag.fillna(0, inplace=True)

# create terms as indices
tf_matrix_diag.set_index(['index_terms_diag'], inplace=True)

# fill tf_matrix with the frequency of each term for every docs
count = 0
for j in word_list_diag:
  count += 1
  for k in df_content.index:
    diag = df_content['FS_DIAGNOSA'][k] 
    docid = k
    f = diag.count(j)
    if f !=0:
      tf_matrix_diag['doc_'+str(docid)][j] = f
  print("done for word:",j, "at level", count)
print("Weighted term frequency matrix completed.") 

done for word: febris at level 1
done for word: hari at level 2
done for word: isk at level 3
done for word: post at level 4
done for word: rawat at level 5
done for word: inap at level 6
done for word: isk at level 7
done for word: post at level 8
done for word: rawat at level 9
done for word: inap at level 10
done for word: obs at level 11
done for word: febris at level 12
done for word: ec at level 13
done for word: kontrol at level 14
done for word: bkb at level 15
done for word: alergi at level 16
done for word: rfa at level 17
done for word: bkb at level 18
done for word: ec at level 19
done for word: alergi at level 20
done for word: susp at level 21
done for word: asma at level 22
done for word: scarlet at level 23
done for word: fever at level 24
done for word: asma at level 25
done for word: bronkhiale at level 26
done for word: serang at level 27
done for word: gigi at level 28
done for word: non at level 29
done for word: vital at level 30
done for word: gingival at level 3

In [48]:
tf_matrix_diag

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_990,doc_991,doc_992,doc_993,doc_994,doc_995,doc_996,doc_997,doc_998,doc_999
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hari,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
isk,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
post,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rawat,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i,2,2,2,1,1,0,1,0,1,5,...,3,3,0,3,1,1,4,1,4,3
vaksin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
hiperemi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [51]:
for i in range(0,N):
    tf_matrix_diag['doc_'+str(i)] = tf_matrix_diag['doc_'+str(i)] / max(tf_matrix_diag['doc_'+str(i)])
    
# append a column D at the end of tf
sLength = len(tf_matrix_diag.index)
D = pd.Series(np.zeros(sLength))
tf_matrix_diag = tf_matrix_diag.assign(D=D.values)
# append an IDF column
IDF = pd.Series(np.zeros(sLength))
tf_matrix_diag = tf_matrix_diag.assign(IDF=IDF.values)
# fill the nonzero count at D

In [52]:
tf_matrix_diag

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_992,doc_993,doc_994,doc_995,doc_996,doc_997,doc_998,doc_999,D,IDF
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,0.5,0.000000,0.000000,0.5,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
hari,0.5,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
isk,0.0,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
post,0.0,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
rawat,0.0,0.333333,0.333333,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i,1.0,0.666667,0.666667,0.5,0.5,0.0,0.333333,0.0,0.25,1.0,...,0.0,1.0,0.25,0.333333,1.0,1.0,1.00,1.0,0.0,0.0
vaksin,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,1.0,0.00,0.0,0.0,0.0
hiperemi,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.25,0.0,0.0,0.0


In [53]:
"""
    computing term frequecny and inverce-document frequency
    for every index terms. two columns are being added for this purpose.
"""
for indices in range(len(tf_matrix_diag.index)):
  di = sum(tf_matrix_diag.loc[tf_matrix_diag.index[indices]] > 0) 
  # update the di values |di|=non zero entries of a term w.r.t a doc
  tf_matrix_diag['D'][tf_matrix_diag.index[indices]] = di
  # IDF computation
  tf_matrix_diag['IDF'][tf_matrix_diag.index[indices]] = math.log(N/di,10)
    
"""
    tf-idf computation phase. compute tf-idf for each entry of the data 
"""
global B
B = tf_matrix_diag
for i in range(len(tf_matrix_diag.index)):
  B.loc[B.index[i]][0:N+1] = B.loc[B.index[i]][0:N+1] * B.IDF[i]

In [54]:
B

,doc_0,doc_1,doc_2,doc_3,doc_4,doc_5,doc_6,doc_7,doc_8,doc_9,...,doc_992,doc_993,doc_994,doc_995,doc_996,doc_997,doc_998,doc_999,D,IDF
index_terms_diag,,,,,,,,,,,,,,,,,,,,,
febris,1.022879,0.000000,0.000000,1.022879,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,18.411817,2.045757
hari,1.110924,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,13.331092,2.221849
isk,0.000000,0.628686,0.628686,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,24.518736,1.886057
post,0.000000,0.331893,0.331893,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,100.563541,0.995679
rawat,0.000000,0.840960,0.840960,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,7.568636,2.522879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i,0.126679,0.084453,0.084453,0.063340,0.06334,0.0,0.042226,0.0,0.03167,0.126679,...,0.0,0.126679,0.03167,0.042226,0.126679,0.126679,0.126679,0.126679,94.629510,0.126679
vaksin,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,3.000000,0.000000,0.000000,3.000000,3.000000
hiperemi,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.750000,0.000000,3.000000,3.000000


# User Query

In [55]:
global K
K = 10

Anamnesa

In [56]:
def user_query_anam_processing(query): 
  q_mat = pd.DataFrame(columns={'index_terms','q'})
  q_mat.index_terms = word_anam
  q_mat.fillna(0,inplace=True)
  q_mat.set_index('index_terms', inplace=True)
          
  # intersect query and index terms
  valid_q = list(set(query).intersection(list(A.index)))
          
  for m in range(len(valid_q)):
    for n in range(len(A)):
      if valid_q[m] == A.index[n]:
        q_mat.q[n] = 1
              
  # print(q_mat.q, A.IDF)            
  q_mat.q = q_mat.q * A.IDF
  # #print(q_mat.q)
  scores = np.zeros(N)
  # # the first position is not needed, so keeping an invalid value there
  scores[0] = -5555555
          
  # """
  #  cosine similarity function:
  #  if any of the denominator or numerator is zero the score is set to 0
  # """
  for i in range(0,N):
    vec_dot = np.dot(q_mat.q, A['doc_'+str(i)])
    vec_norm = (LA.norm(q_mat.q) * LA.norm(A['doc_'+str(i)]))
    if vec_dot == 0 or vec_norm == 0:
      scores[i] = 0.0
    if vec_dot > 0 or vec_norm > 0:
      scores[i] = vec_dot / vec_norm

  global top_scores, top_docs 
  
  top_scores = sorted(scores,reverse=True)
  top_n_scores = top_scores[:K]
  top_docs = np.array(np.argsort(scores))
  top_docs = top_docs[::-1] # reverse
  top_n_docs = top_docs[:K]

  global ret_doc_id
  ret_doc_id = pd.DataFrame(columns=['Ranking', 'Scores', 'Document Number', 'FS_ANAMNESA', 'FS_TINDAKAN', 'FS_TERAPI', 'FS_CATATAN_FISIK',
        'FS_DIAGNOSA'])
  
  print('Rank'+'\t'+'Scores'+'\t' +'Document number '+ '\t'+ 'ANAMNESA '+'\t' + 'TINDAKAN '+'\t' + 'TERAPI '+'\t' + 'CATATAN FISIK '+'\t' + 'DIAGNOSA'+'\n' )
  for p in range(K):
    ret_doc_id = ret_doc_id.append({'Ranking': p+1, 'Scores': round(top_n_scores[p],3), 'Document Number': top_n_docs[p], 'FS_ANAMNESA': data.loc[top_n_docs[p]].FS_ANAMNESA, 'FS_TINDAKAN': data.loc[top_n_docs[p]].FS_TINDAKAN, 'FS_TERAPI': data.loc[top_n_docs[p]].FS_TERAPI, 'FS_CATATAN_FISIK': data.loc[top_n_docs[p]].FS_CATATAN_FISIK, 'FS_DIAGNOSA': data.loc[top_n_docs[p]].FS_DIAGNOSA}, ignore_index=True)
    ret_doc_id.set_index('Ranking', inplace = True)
  
  # global con_ret_doc_id
  # con_ret_doc_id = con_ret_doc_id.append(ret_doc_id, ignore_index=True)

  ret_doc_id.set_index('Ranking', inplace = True)
  display(ret_doc_id)

Diagnosa

In [57]:
def user_query_diag_processing(query): 
  q_mat = pd.DataFrame(columns={'index_terms','q'})
  q_mat.index_terms = word_diag
  q_mat.fillna(0,inplace=True)
  q_mat.set_index('index_terms', inplace=True)
          
  # intersect query and index terms
  valid_q = list(set(query).intersection(list(B.index)))
          
  for m in range(len(valid_q)):
    for n in range(len(B)):
      if valid_q[m] == B.index[n]:
        q_mat.q[n] = 1
              
  # print(q_mat.q, A.IDF)            
  q_mat.q = q_mat.q * B.IDF
  # #print(q_mat.q)
  scores = np.zeros(N)
  # # the first position is not needed, so keeping an invalid value there
  scores[0] = -5555555
          
  # """
  #  cosine similarity function:
  #  if any of the denominator or numerator is zero the score is set to 0
  # """

  for i in range(0,N):
    vec_dot = np.dot(q_mat.q, B['doc_'+str(i)])
    vec_norm = (LA.norm(q_mat.q) * LA.norm(B['doc_'+str(i)]))
    if vec_dot == 0 or vec_norm == 0:
      scores[i] = 0.0
    if vec_dot > 0 or vec_norm > 0:
      scores[i] = vec_dot / vec_norm 

  global top_scores, top_docs 

  top_scores = sorted(scores,reverse=True)
  top_n_scores = top_scores[:K]
  top_docs = np.array(np.argsort(scores))
  top_docs = top_docs[::-1] # reverse
  top_n_docs = top_docs[:K]

  global ret_doc_id
  ret_doc_id = pd.DataFrame(columns=['Ranking', 'Scores', 'Document Number', 'FS_ANAMNESA', 'FS_TINDAKAN', 'FS_TERAPI', 'FS_CATATAN_FISIK',
        'FS_DIAGNOSA'])
  
  print('Rank'+'\t'+'Scores'+'\t' +'Document number '+ '\t'+ 'ANAMNESA '+'\t' + 'TINDAKAN '+'\t' + 'TERAPI '+'\t' + 'CATATAN FISIK '+'\t' + 'DIAGNOSA'+'\n' )
  for p in range(K):
    ret_doc_id = ret_doc_id.append({'Ranking': p+1, 'Scores': round(top_n_scores[p],3), 'Document Number': top_n_docs[p], 'FS_ANAMNESA': data.loc[top_n_docs[p]].FS_ANAMNESA, 'FS_TINDAKAN': data.loc[top_n_docs[p]].FS_TINDAKAN, 'FS_TERAPI': data.loc[top_n_docs[p]].FS_TERAPI, 'FS_CATATAN_FISIK': data.loc[top_n_docs[p]].FS_CATATAN_FISIK, 'FS_DIAGNOSA': data.loc[top_n_docs[p]].FS_DIAGNOSA}, ignore_index=True)

  ret_doc_id.set_index('Ranking', inplace = True)
  display(ret_doc_id)

In [58]:
def prec(ret):
  rel_items_retrieved = 0
  ret_items = 0
  
  for a in range(1, len(ret)+1):
    ret_items += 1
    if(ret['Scores'].loc[ret.index == a] > 0).all() :
      rel_items_retrieved += 1
    elif(ret['Scores'].loc[ret.index == a] == 0).all() :
      continue

  precision = rel_items_retrieved / ret_items
  return precision

In [59]:
con_precision = [] 

def precision(ret):
  data = prec(ret)
  con_precision.append(data)

Recall

In [60]:
def rec(ret, document):
  rel_items_retrieved = 0
  rel_items = 0

  for a in range(1, len(ret)+1):
    if(ret['Scores'].loc[ret.index == a] > 0).all() :
      rel_items_retrieved += 1
    elif(ret['Scores'].loc[ret.index == a] == 0).all() :
      continue

  for b in range(len(document)):
    if(document[b] != 0):
      rel_items += 1
    elif(document[b] == 0):
      break

  recall = rel_items_retrieved / rel_items
  return recall

In [61]:
con_recall = [] 

def recall(ret, document):
  data = rec(ret, document)
  con_recall.append(data)

Query

In [85]:
print('Apa yang akan Anda pilih?')
print('Anamnesa ketikkan 0')
print('Diagnosa ketikkan 1')

inputan = input("yang Anda pilih adalah ")

while True:
  user_query = word_tokenize(input("Search query >> "))
  if user_query == []:
    sys.exit()
  elif inputan == 0:
    user_query_anam_processing(user_query)
    precision(ret_doc_id)
    recall(ret_doc_id, top_scores)
  else:
    user_query_diag_processing(user_query)
    precision(ret_doc_id)
    recall(ret_doc_id, top_scores)

Apa yang akan Anda pilih?
Anamnesa ketikkan 0
Diagnosa ketikkan 1
yang Anda pilih adalah 1
Search query >> hamil
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNOSA



,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.657,787,"<p>g4p1ab2ah0,hpm tgl 5/10U/19</p>",<p>USG belum tampak GS</p>,<p>R/ Folaplus No XXX S1dd1</p>,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Hamil 5 mg</p>
2,0.657,30,<p>kadang kenceng kenceng.hpm 11 juni 18.uk23+...,"<p>janin tunggal, presbo, laki laki, djj+, ger...",<p>folamil genio no 30</p> <p>S1dd1</p>,"<p>Suhu : 36 C,Nadi : x/menit,Respirasi : 20 x...",<p>hamil 23 mg</p>
3,0.657,34,"<p>kontrol hamil, G4 P2 A1, HPHT=11-6-18</p>",<p>257 gram sesuai hamil 18 mg</p> <p>Px insp...,<p>pregnacare plus no 30</p> <p>S1dd1</p>,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>hamil 18 mg</p>
4,0.657,758,<p>g3p0a2 hpht 30 - 5 -2018 kontrol hamil .. t...,"<p>167 gr sesuai hamil 16 mg, Djj+, gerak +</p>",<p>folamil genio 30</p> <p>s1dd1</p> <p>cavi...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>hamil 17 mg</p>
5,0.657,957,<p>G2 P0 A1 . HPMT 18-4-18 . HPL 25-1-19 .</p>,"<p>preskep, djj+, mgerak +, TBJ 2505 gr, amati...",<p>pregnacare plus no 10</p> <p>s1dd1</p> <p...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>hamil 36 mg</p>
6,0.657,954,"<p>g 2 p1 a0.hpm tgl 18 april 18 uk, 31 + 5mg</p>","<p>janin tunggal, preskep, djj+, gerak +, 1926...",<p>maltofer fol N0 15</p> <p>s1dd1</p> <p>Ca...,"<p>Suhu : 36 C,Nadi : 65 x/menit,Respirasi : 2...",<p>Hamil 31 mg</p>
7,0.657,950,<p>g1p0a0(14/06/18) kell; mual mual. pp test +...,"<p>USG : GS +, fetal pole +, pulsasi +</p>",<p>anvomer B6 no 15</p> <p>S2 dd1 prn</p>,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>hamil 8 mg</p>
8,0.657,765,"<p>G1P0AB0 HPM TGL 13/3/19,TAA</p>","<p>janin tunggal, djj+, gerak + ketuban cukup,...",<p>Osteocare no 10</p> <p>S1dd1</p> <p>Folam...,"<p>Suhu : C,Nadi : 86 x/menit,Respirasi : x/me...",<p>hamil 29 mg</p>
9,0.657,766,<p>g1p0ab0 hpht tgl 13-3-18 dg kel;-</p>,"<p>USG GS + intrauterin</p> <p>GS 18,6 sesuai...",<p>anemolat 1 mg no 20</p> <p>S1dd1</p> <p>O...,"<p>Suhu : C,Nadi : 89 x/menit,Respirasi : x/me...",<p>hamil 6 mg</p>


Search query >> dermatitis
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNOSA



,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.436,98,<p>GATAL DI PAHA makin banyak&nbsp; 1th&nbsp;<...,NaN,<p>R/ Hexilon 8 mg tab no X</p> <p>S 1-0-0&nb...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
2,0.436,97,<p>KONTROL sempat membaik&nbsp;</p>,NaN,<p>R/ Hexilon 8 mg tab no XV</p> <p>S 1-0-0&n...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
3,0.436,806,<p>-gatal di kaki</p>,NaN,<p>R/&nbsp; IKADERM CREAM gr 10&nbsp;<br /> BA...,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>Dermatitis Numular&nbsp;</p>
4,0.436,805,<p>gatal di kaki kambuh&nbsp;</p>,NaN,<p>R/&nbsp; IKADERM CREAM gr 10&nbsp;<br /> BA...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
5,0.436,804,<p>gatal2 dikaki sdh 1 bln</p>,NaN,<p>R/&nbsp; IKADERM CREAM gr 10&nbsp;<br /> BA...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
6,0.436,143,<p>gatal di tangan dan kaki kambuh&nbsp;</p>,NaN,<p>R/ HEXILON 8MG&nbsp;&nbsp;tab no X</p> <p>...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
7,0.436,301,<p>gatal2 di kaki&nbsp;</p>,NaN,<p>R/ Desoxymetason gr 15&nbsp;</p> <p>Fuladi...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Numular&nbsp;</p>
8,0.410,207,<p>GATAL DI KAKI</p>,NaN,<p>R/&nbsp;TERMISIL CR&nbsp;&nbsp;gr 10&nbsp;<...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Atopik&nbsp;</p> <p>&nbsp;Derma...
9,0.399,246,"<p>KONTROL, membaik</p>",NaN,<p>R/ Metylprednisolon 8 mg tab no X&nbsp;</p>...,"<p>Suhu : C,Nadi : x/menit,Respirasi : x/menit...",<p>Dermatitis Atopik&nbsp;</p>


Search query >> et
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNOSA



...

Search query >> meningioma
Rank	Scores	Document number 	ANAMNESA 	TINDAKAN 	TERAPI 	CATATAN FISIK 	DIAGNOSA



,Scores,Document Number,FS_ANAMNESA,FS_TINDAKAN,FS_TERAPI,FS_CATATAN_FISIK,FS_DIAGNOSA
Ranking,,,,,,,
1,0.413,789,<p>kontrol rutin</p>,NaN,<p>&nbsp;</p> <p>Mecobalamine 2 x 1 tab&nbsp;...,"<p>Suhu : C,Nadi : 69 x/menit,Respirasi : x/me...",<p>Pst Craniotomi Meningioma</p>
2,0.413,790,<p>KONTROL RUTIN</p> <p>gangguan pikiran</p>,NaN,<p>Haloperidol 2 mg&nbsp; &nbsp; &nbsp; 1x1 ta...,"<p>Suhu : C,Nadi : 67 x/menit,Respirasi : x/me...",<p>Pst Craniotomi Meningioma</p>
3,0.413,796,<p>kontrol rutin</p>,NaN,<p>&nbsp;</p> <p>Mecobalamine 2 x 1 tab&nbsp;...,"<p>Suhu : C,Nadi : 82 x/menit,Respirasi : x/me...",<p>Pst Craniotomi Meningioma</p>
4,0.351,791,"<p>kontrol. Nyeri kepala (-), muntah (-). nyer...",<p>-</p>,<p>Mecobalamine 2 x 1 tab&nbsp; &nbsp; &nbsp; ...,"<p>Suhu : C,Nadi : 77 x/menit,Respirasi : x/me...",<p>post craniotomy meningioma</p> <p>psiko ne...
5,0.351,795,<p>kontrol</p>,NaN,<p>Mecobalamine 2 x 1 tab&nbsp; &nbsp; &nbsp; ...,"<p>Suhu : C,Nadi : 65 x/menit,Respirasi : x/me...",<p>post craniotomy meningioma</p> <p>psiko ne...
6,0.351,788,<p>KONTROL</p>,NaN,<p>&nbsp;</p> <p>Mecobalamine&nbsp; XX&nbsp; ...,"<p>Suhu : C,Nadi : 69 x/menit,Respirasi : x/me...",<p>post craniotomy meningioma</p> <p>psiko ne...
7,0.351,968,<p>KONTROL</p>,NaN,<p>&nbsp;</p> <p>Mecobalamine&nbsp; XX&nbsp; ...,"<p>Suhu : C,Nadi : 65 x/menit,Respirasi : x/me...",<p>post craniotomy meningioma</p> <p>psiko ne...
8,0.351,969,<p>-kontrol post craniotomy meningioma psiko n...,NaN,<p>&nbsp;</p> <p>Lansoprazole&nbsp; &nbsp; &n...,"<p>Suhu : - C,Nadi : -71 x/menit,Respirasi : -...",<p>post craniotomy meningioma</p> <p>psiko ne...
9,0.351,970,<p>-kontrol post craniotomy meningioma psiko n...,NaN,<p>&nbsp;</p> <p>Lansoprazole&nbsp; &nbsp; &n...,"<p>Suhu : - C,Nadi : - x/menit,Respirasi : - x...",<p>post craniotomy meningioma</p> <p>psiko ne...


Search query >> 


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Evaluation

## Precision

In [86]:
def avg_precision(data):
  jumlah = sum(data)
  panjang = len(data)

  hasil = jumlah / panjang
  return hasil

In [87]:
avg_precision(con_precision)

0.5739130434782608

## Recall

In [88]:
def avg_recall(data):
  jumlah = sum(data)
  panjang = len(data)

  hasil = jumlah / panjang
  return hasil

In [89]:
avg_recall(con_recall)

0.7809222305533547